In [146]:
from bs4 import BeautifulSoup as bs
import urllib2
import re
import csv
from collections import OrderedDict
import cookielib
import time

In [202]:
start = 'http://archiveofourown.org/tags/James%20Bond*s*Q/works'

In [203]:
#testing cookie usage

In [204]:
#get a cookie for going through content filtering
cookie = cookielib.CookieJar()
opener = urllib2.build_opener(urllib2.HTTPCookieProcessor(cookie))
opener.open('http://archiveofourown.org/works/5051548?view_full_work=true')

<addinfourl at 4420434112 whose fp = <socket._fileobject object at 0x1062817d0>>

In [205]:
req = urllib2.Request("http://archiveofourown.org/works/5051548?view_adult=true")
opener2 = urllib2.build_opener(urllib2.HTTPCookieProcessor(cookie))
page = opener2.open(req)

In [206]:
cookie

<CookieJar[Cookie(version=0, name='_otwarchive_session', value='BAh7CUkiD3Nlc3Npb25faWQGOgZFVEkiJTIwMWYyYjdiYjUxZGIzNjhhMzMwNGRiZjFjZDUwZWY0BjsAVEkiDnJldHVybl90bwY7AEYiJS93b3Jrcy81MDUxNTQ4L2NoYXB0ZXJzLzExNjE2NDI3SSIQX2NzcmZfdG9rZW4GOwBGSSIxRlJtQlp0aWFtTVI3OWJEZHNDbEJpSjRSdlZUY05sUXhuaDhSeXFDOWFpQT0GOwBGSSIKYWR1bHQGOwBGVA%3D%3D--a608fa5d4251a52193c2e3c77dfe507e28b3c593', port=None, port_specified=False, domain='archiveofourown.org', domain_specified=False, domain_initial_dot=False, path='/', path_specified=True, secure=False, expires=None, discard=True, comment=None, comment_url=None, rest={'HttpOnly': None}, rfc2109=False)]>

In [207]:
def find_page(base_url, page_number):
    #go to any page number.
    return base_url+'?page=' +str(page_number)

In [208]:
def find_works(page):
    #Find all works from a works list page.
    works_page = bs(urllib2.urlopen(page))
    links = []
    for link in works_page.find_all('a'):
        url = link.get('href')
        url_s = [i for i in url.split('/') if i != '']
        if 'work' in url and len(url_s) == 2 and str(url_s[1]).isdigit():
                links.append('http://archiveofourown.org'+link.get('href'))
    return links

In [209]:
def show_full_contents(url):
    #go through adult contents filtering.
    base = bs(urllib2.urlopen(url))
    full_url = url
    for link in base.find_all('a'):
        if 'Proceed' in link.text:
            full_url = url +'?view_adult=true'

#     can't show full work in the same way.
#     base2 = bs(urllib2.urlopen(full_url))
#     print full_url
#     for link in base2.find_all('a'):
#         if 'Entire Work' in link.text:
#             full_url += '?view_full_work=true'

    return full_url

In [210]:
def get_next_chapter(url):
    #for multi chapter works, get next chapter link.
    req = urllib2.Request(url)
    opener = urllib2.build_opener(urllib2.HTTPCookieProcessor(cookie))
    page = bs(opener2.open(req))
    next_chapter = ''
    for link in page.find_all('a'):
        if 'Next Chapter' in link.text:
            next_chapter = ('http://archiveofourown.org' + link.get('href'))
    return next_chapter

In [211]:
def get_download_link(url):
    #may use the download page alternatively, where the html page is cleaner. But this misses some features, so not use
    #at the moment.
    req = urllib2.Request(url)
    opener = urllib2.build_opener(urllib2.HTTPCookieProcessor(cookie))
    page = bs(opener2.open(req))
    for link in page.find_all('a'):
        if 'HTML' in link.text:
            download_link = 'http://archiveofourown.org' + link.get('href')
    return download_link

In [212]:
get_next_chapter('http://archiveofourown.org/works/3078407?view_adult=true')

'http://archiveofourown.org/works/3078407/chapters/9335556'

In [213]:
# get_contents('http://archiveofourown.org/works/5051548?view_full_work=true')

In [214]:
#unfinished parser for download page, see above.
# def create_work_dict_dlpage(url, contents):
#     #get work metadata and contents into a dictrionary.
#     print 'Getting work information from:', url
# #     try:
#     work = {}
    
#     work['Rating'] = str(re.findall('Rating:ANot Rated|Rating:AGeneral Audiences|Rating:ATeen And Up Audiences|Rating:AMature|Rating:AExplicit',contents)[0]).strip()
#     work['Archive Warning'] = re.findall('Archive Warning:ACreator Chose Not To Use Archive Warnings|Archive Warning:AGraphic Depictions Of Violence|Archive Warning:AMajor Character Death|Archive Warning:ANo Archive Warnings Apply|Archive Warning:ARape/Non-Con|Archive Warning:AUnderage',contents)[0].strip()
#     work['Fandoms'] = [i for i in re.findall('Fandoms:A(.*?)A|Fandom:A(.*?)A',contents)[0] if i != ''][0].strip()
    
#     category = re.findall('Category:AF/F|Category:AF/M|Category:AGen|Category:AM/M|Category:AMulti|Category:AOther',contents)[0].strip()
#     if category == []:
#         work['Category'] = ''
#     else:
#         work['Category'] = category[0].strip()
    
#     relationship = re.findall('Relationship:A(.*?)A',contents)
#     if relationship == []:
#         work['Relationship'] = ''
#     else:
#         work['Relationship'] = relationship[0].strip()
        
#     characters = re.findall('Characters:A(.*?)A',contents)
#     if characters == []:
#         work['Characters'] = ''
#     else:
#         work['Characters'] = characters[0].strip()
        
#     additional = re.findall('Additional Tags:A(.*?)A',contents)
#     if additional == []:
#         work['Additional Tags'] = ''
#     else:
#         work['Additional Tags'] = additional[0].strip()
    
#     language = re.findall('Language:A      AA(.*?)A      A',contents)
#     if language == []:
#         work['Language'] = ''
#     else:
#         work['Language'] = language[0].strip()
    
#     stats = re.findall('Stats:AAA(.*?)AAAAAAAA',contents)
#     if stats == []:
#         work['Stats'] = ''
#     else:
#         work['Stats'] = stats[0].strip()
    
#     author = re.findall('A    AA(.*?)AAA',contents)
#     if author == []:
#         work['Author'] = ''
#     else:
#         work['Author'] = author[0].strip()
    
#     text = re.findall('Work Text:(.*?)AAAAAAAA|Chapter TextA(.*?)AAAAA',contents)
#     if text == []:
#         work['Test'] = ''
#     else:
#         work['Text']= [i for i in text[0] if i != ''][0].strip()
    
#     title = re.findall('AAAAAAAA      (.*?)A    AA',contents)
#     if title == []:
#         work['Title'] = ''
#     else:
#         work['Title']  = title[0].strip()
    
#     summary = re.findall('Summary:AAA(.*?)AAAAA',contents)
#     if summary == []:
#         work['Summary'] = ''
#     else:
#         work['Summary'] = summary[0].strip()

        
#     notes = re.findall('Notes:AA(.*?)AA',contents)
#     if notes == []:
#         work['Notes'] = ''
#     else:
#         work['Notes'] = notes[0].strip()
        
#     print 'Finished with:', url
# #     except:
# #         print 'Something went wrong.'
#     return work

In [215]:
def get_contents(url):
    #get work metadata and contents from the work page.
#     print 'Reading url:', url
    try:
        req = urllib2.Request(url)
        opener = urllib2.build_opener(urllib2.HTTPCookieProcessor(cookie))
        page = bs(opener2.open(req))
        contents = str(page.body.text.encode('utf-8')).replace('\n','A')
    except:
        print 'Unable to read from this url'
        contents = ''
        with open('/Users/jingy/Desktop/problematic_url.csv', 'a') as g:
            g.write(url)
    return url, contents

In [216]:
# get_contents('http://archiveofourown.org/works/5051548?view_full_work=true')

In [217]:
def write_header():
    f = open('/Users/jingy/Desktop/works.csv', 'a')
    writer = csv.writer(f, delimiter=',')
    keys = ['Additional Tags', 'Archive Warning', 'Author', 'Category', 'Characters', 'Fandoms', 'Language', 'Notes',\
            'Rating', 'Relationship', 'Stats', 'Summary', 'Text', 'Title']
    writer.writerow(keys)
    f.close()

In [218]:
def write_work_content(work_dict):
    #write work metadata and contents as values of a sorted dictionary.
    f = open('/Users/jingy/Desktop/works.csv', 'a')
    writer = csv.writer(f, delimiter=',')
    try:
        writer.writerow(OrderedDict(sorted(work_dict.items())).values())
    except:
        pass
    f.close()

In [219]:
#creates dictionary for information in a single work.
def create_work_dict(url, contents):
    #get work metadata and contents into a dictionary.
    #Know it's not elegent...
#     print 'Getting work information from:', url
    try:
        work = {}

        work['Rating'] = str(re.findall('Not Rated|General Audiences|Teen And Up Audiences|Mature|Explicit',contents)[0]).strip()
        work['Archive Warning'] = re.findall('Creator Chose Not To Use Archive Warnings|Graphic Depictions Of Violence|Major Character Death|No Archive Warnings Apply|Rape/Non-Con|Underage',contents)[0].strip()
        work['Fandoms'] = [i for i in re.findall('Fandoms:A          AAA(.*?)AAAA|Fandom:A          AAA(.*?)AAAA',contents)[0] if i != ''][0]

        category = re.findall('Category:F/F|Category:F/M|Category:Gen|Category:M/M|Category:Multi|Category:Other',contents)
        if category == []:
            work['Category'] = ''
        else:
            work['Category'] = category[0].strip()

        relationship = re.findall('Relationship:A          AAA(.*?)AAAA',contents)
        if relationship == []:
            work['Relationship'] = ''
        else:
            work['Relationship'] = relationship[0].strip()

        characters = re.findall('Characters:A          AAA(.*?)AAAA',contents)
        if characters == []:
            work['Characters'] = ''
        else:
            work['Characters'] = characters[0].strip()

        additional = re.findall('Additional Tags:A          AAA(.*?)AAAA',contents)
        if additional == []:
            work['Additional Tags'] = ''
        else:
            work['Additional Tags'] = additional[0].strip()

        language = re.findall('Language:A      AA(.*?)A      A',contents)
        if language == []:
            work['Language'] = ''
        else:
            work['Language'] = language[0].strip()

        stats = re.findall('Stats:AAA(.*?)AAAAAAAA',contents)
        if stats == []:
            work['Stats'] = ''
        else:
            work['Stats'] = stats[0].strip()

        author = re.findall('A    AA(.*?)AAA',contents)
        if author == []:
            work['Author'] = ''
        else:
            work['Author'] = author[0].strip()

        text = re.findall('Work Text:(.*?)AAAAAAAA|Chapter TextA(.*?)AAAAA',contents)
        if text == []:
            work['Test'] = ''
        else:
            work['Text']= [i for i in text[0] if i != ''][0].strip()

        title = re.findall('AAAAAAAA      (.*?)A    AA',contents)
        if title == []:
            work['Title'] = ''
        else:
            work['Title']  = title[0].strip()

        summary = re.findall('Summary:AAA(.*?)AAAAA',contents)
        if summary == []:
            work['Summary'] = ''
        else:
            work['Summary'] = summary[0].strip()


        notes = re.findall('Notes:AA(.*?)AA',contents)
        if notes == []:
            work['Notes'] = ''
        else:
            work['Notes'] = notes[0].strip()
        
#     print 'Finished with:', url
    except:
        print 'Something went wrong.'
    return work

In [220]:
# d = create_work_dict('http://archiveofourown.org/works/5205566',str(c))

In [221]:
# c = bs(urllib2.urlopen('http://archiveofourown.org/works/3078407/chapters/6678515')).body.text

In [222]:
def read_single_work(url):
    url_full = show_full_contents(url)
    u, c = get_contents(url_full)
    work = create_work_dict(u, c)
    write_work_content(work)

In [223]:
def get_chapters(url):
    url_full = show_full_contents(url)
    chapters_list = []
    next_url = get_next_chapter(url_full)
    while next_url != '':
        chapters_list.append(next_url)
        next_url = get_next_chapter(next_url)
    return chapters_list

In [224]:
# get_chapters('http://archiveofourown.org/works/3078407?view_adult=true')

In [ ]:
write_header()
start_time = time.clock()

for i in range(1,10):
    page = find_page(start, i)
    print 'crawling page:', i
    worklist = find_works(page)
    for w in worklist:
        print 'reading work:', w
        ch_list = get_chapters(w)
        if ch_list != []:
            read_single_work(w)
            for ch in ch_list:
                print 'reading chapter:', ch
                read_single_work(ch)
        else:
            read_single_work(w)
        
print 'Retrived %s works from the first %s pages of %s in %s seconds .' %(len(worklist), i, 'James Bond/Q', str(time.clock() - start_time))        

crawling page: 1
reading work: http://archiveofourown.org/works/3078407
reading chapter: http://archiveofourown.org/works/3078407/chapters/9335556
reading chapter: http://archiveofourown.org/works/3078407/chapters/12008975
reading work: http://archiveofourown.org/works/5209502
reading work: http://archiveofourown.org/works/5051548
reading chapter: http://archiveofourown.org/works/5051548/chapters/11662546
reading chapter: http://archiveofourown.org/works/5051548/chapters/11715200
reading chapter: http://archiveofourown.org/works/5051548/chapters/11766752
reading chapter: http://archiveofourown.org/works/5051548/chapters/11840177
reading chapter: http://archiveofourown.org/works/5051548/chapters/11886449
reading chapter: http://archiveofourown.org/works/5051548/chapters/11941022
reading chapter: http://archiveofourown.org/works/5051548/chapters/12007541
reading work: http://archiveofourown.org/works/5086573
reading chapter: http://archiveofourown.org/works/5086573/chapters/11696602
read